## 模型设计

In [2]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import operator
from matplotlib import pylab as plt
from datetime import datetime
import time
from sklearn.model_selection import GridSearchCV

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [143]:
data = pd.read_csv('train_set.csv')
data.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,label
0,202633.0,12564.0,8.0,3.0,0.0,0.0,0.0,0.0,6.0,3.0,...,62.6,15502.8,1.0,0.0260,0.0,0.0,0.0,0.0,1.0,1.0
1,218498.0,149854.0,8.0,12.0,0.0,0.0,0.0,0.0,12.0,6.0,...,62.6,15502.8,1.0,0.0403,0.0,0.0,0.0,0.0,1.0,1.0
2,221842.0,75877.0,8.0,9.0,0.0,0.0,0.0,0.0,15.0,237.0,...,62.6,15502.8,1.0,0.0245,0.0,0.0,0.0,0.0,1.0,1.0
3,222886.0,154636.0,8.0,60.0,3.0,0.0,0.0,0.0,78.0,30.0,...,62.6,15502.8,1.0,0.0208,0.0,0.0,0.0,0.0,1.0,1.0
4,235240.0,38222.0,8.0,90.0,3.0,0.0,0.0,0.0,84.0,165.0,...,62.6,15502.8,1.0,0.0166,0.0,0.0,0.0,0.0,1.0,1.0


In [144]:
data.columns

Index(['user_id', 'sku_id', 'cate', 'action_before_3_1.0_x',
       'action_before_3_2.0_x', 'action_before_3_3.0_x',
       'action_before_3_4.0_x', 'action_before_3_5.0_x',
       'action_before_3_6.0_x', 'action_before_3_1.0_y',
       ...
       'cate_action_5_mean', 'cate_action_6_mean', 'has_bad_comment',
       'bad_comment_rate', 'comment_num_0', 'comment_num_1', 'comment_num_2',
       'comment_num_3', 'comment_num_4', 'label'],
      dtype='object', length=251)

In [145]:
data_x = data.loc[:,data.columns != 'label']
data_y = data.loc[:,data.columns == 'label']

In [146]:
data_x.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
0,202633.0,12564.0,8.0,3.0,0.0,0.0,0.0,0.0,6.0,3.0,...,25.2,62.6,15502.8,1.0,0.0260,0.0,0.0,0.0,0.0,1.0
1,218498.0,149854.0,8.0,12.0,0.0,0.0,0.0,0.0,12.0,6.0,...,25.2,62.6,15502.8,1.0,0.0403,0.0,0.0,0.0,0.0,1.0
2,221842.0,75877.0,8.0,9.0,0.0,0.0,0.0,0.0,15.0,237.0,...,25.2,62.6,15502.8,1.0,0.0245,0.0,0.0,0.0,0.0,1.0
3,222886.0,154636.0,8.0,60.0,3.0,0.0,0.0,0.0,78.0,30.0,...,25.2,62.6,15502.8,1.0,0.0208,0.0,0.0,0.0,0.0,1.0
4,235240.0,38222.0,8.0,90.0,3.0,0.0,0.0,0.0,84.0,165.0,...,25.2,62.6,15502.8,1.0,0.0166,0.0,0.0,0.0,0.0,1.0


In [147]:
data_y.head()

,label
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [148]:
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size = 0.2, random_state = 0)

In [149]:
x_test.shape

(2924, 250)

In [150]:
x_val = x_test.iloc[:1500,:]
y_val = y_test.iloc[:1500,:]

x_test = x_test.iloc[1500:,:] 
y_test = y_test.iloc[1500:,:]

In [151]:
print (x_val.shape)
print (x_test.shape)

(1500, 250)
(1424, 250)


In [152]:
del x_train['user_id']
del x_train['sku_id']

del x_val['user_id']
del x_val['sku_id']

x_train.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
2157,8.0,6.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,...,41.2,181.3,39138.4,1.0,0.0550,0.0,0.0,0.0,0.0,1.0
2464,9.0,24.0,6.0,0.0,0.0,0.0,54.0,60.0,6.0,0.0,...,11.2,41.2,9593.4,1.0,0.0453,0.0,0.0,0.0,0.0,1.0
10326,9.0,6.0,0.0,0.0,0.0,0.0,15.0,360.0,9.0,0.0,...,35.9,109.0,24141.4,1.0,0.0541,0.0,0.0,0.0,0.0,1.0
7025,4.0,6.0,0.0,0.0,0.0,0.0,6.0,468.0,15.0,9.0,...,129.0,289.9,84741.8,1.0,0.0435,0.0,0.0,0.0,1.0,0.0
6625,4.0,3.0,0.0,0.0,0.0,0.0,18.0,24.0,0.0,0.0,...,129.0,289.9,84741.8,1.0,0.0249,0.0,0.0,0.0,0.0,1.0


In [153]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val)

In [154]:
param = {'n_estimators': 4000, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0, 'subsample': 1.0, 
             'colsample_bytree': 0.8, 'scale_pos_weight':10, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic',
             'eval_metric':'auc'}

In [155]:
num_round = param['n_estimators']

plst = param.items()
evallist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=10)
bst.save_model('bst.model')


[0]	train-auc:0.938692	eval-auc:0.931439
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 10 rounds.
[1]	train-auc:0.948637	eval-auc:0.942691
[2]	train-auc:0.94976	eval-auc:0.942787
[3]	train-auc:0.955299	eval-auc:0.950353
[4]	train-auc:0.957374	eval-auc:0.951207
[5]	train-auc:0.95833	eval-auc:0.951643
[6]	train-auc:0.958296	eval-auc:0.951526
[7]	train-auc:0.958515	eval-auc:0.952207
[8]	train-auc:0.958798	eval-auc:0.951574
[9]	train-auc:0.958875	eval-auc:0.952843
[10]	train-auc:0.960028	eval-auc:0.952843
[11]	train-auc:0.960172	eval-auc:0.952585
[12]	train-auc:0.960095	eval-auc:0.951672
[13]	train-auc:0.960522	eval-auc:0.95217
[14]	train-auc:0.960902	eval-auc:0.952196
[15]	train-auc:0.96096	eval-auc:0.952236
[16]	train-auc:0.96155	eval-auc:0.952651
[17]	train-auc:0.961637	eval-auc:0.953367
[18]	train-auc:0.961876	eval-auc:0.953029
[19]	train-auc:0.962398	eval-auc:0.953497
[20]	train-auc:0.962735	eval-auc:0

In [52]:
print (bst.attributes())

{'best_iteration': '255', 'best_score': '0.974271', 'best_msg': '[255]\ttrain-auc:0.992115\teval-auc:0.974271'}


In [193]:
def create_feature_map(features):
    outfile = open(r'xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()


features = list(x_train.columns[:])
create_feature_map(features)

In [194]:
def feature_importance(bst_xgb):
    importance = bst_xgb.get_fscore(fmap=r'xgb.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)

    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    file_name = 'feature_importance_' + str(datetime.now().date())[5:] + '.csv'
    df.to_csv(file_name)

feature_importance(bst)

In [202]:
fi = pd.read_csv('feature_importance_10-24.csv')
fi.sort_values("fscore", inplace=True, ascending=False)
fi.head()

,Unnamed: 0,feature,fscore
0,0,brand,0.039548
1,1,bad_comment_rate,0.032486
2,2,user_action_30_2_ratio,0.027542
3,3,action_before_7_5.0_x,0.026130
4,4,product_action_2_ratio,0.022599


In [156]:
x_test.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
6765,289984.0,70167.0,4.0,15.0,3.0,0.0,0.0,0.0,36.0,21.0,...,129.0,289.9,84741.8,1.0,0.0615,0.0,0.0,0.0,0.0,1.0
13767,291939.0,16139.0,9.0,6.0,0.0,0.0,0.0,0.0,15.0,138.0,...,47.5,140.6,31869.3,0.0,0.0000,0.0,0.0,1.0,0.0,0.0
9672,288632.0,22140.0,6.0,12.0,0.0,0.0,0.0,0.0,21.0,0.0,...,114.7,264.0,52674.1,1.0,0.0325,0.0,0.0,0.0,0.0,1.0
9116,267503.0,154636.0,8.0,12.0,0.0,0.0,0.0,0.0,15.0,96.0,...,97.2,418.3,90099.3,1.0,0.0209,0.0,0.0,0.0,0.0,1.0
10055,258492.0,39425.0,8.0,3.0,0.0,0.0,0.0,0.0,0.0,21.0,...,105.7,447.8,96897.7,1.0,0.0300,0.0,0.0,0.0,0.0,1.0


In [157]:
users = x_test[['user_id', 'sku_id', 'cate']].copy()
del x_test['user_id']
del x_test['sku_id']
x_test_DMatrix = xgb.DMatrix(x_test)
y_pred = bst.predict(x_test_DMatrix, ntree_limit=bst.best_ntree_limit)

In [158]:
x_test['pred_label'] = y_pred
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label
6765,4.0,15.0,3.0,0.0,0.0,0.0,36.0,21.0,0.0,6.0,...,289.9,84741.8,1.0,0.0615,0.0,0.0,0.0,0.0,1.0,0.000244
13767,9.0,6.0,0.0,0.0,0.0,0.0,15.0,138.0,0.0,0.0,...,140.6,31869.3,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.000006
9672,6.0,12.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,...,264.0,52674.1,1.0,0.0325,0.0,0.0,0.0,0.0,1.0,0.001599
9116,8.0,12.0,0.0,0.0,0.0,0.0,15.0,96.0,3.0,0.0,...,418.3,90099.3,1.0,0.0209,0.0,0.0,0.0,0.0,1.0,0.754251
10055,8.0,3.0,0.0,0.0,0.0,0.0,0.0,21.0,3.0,0.0,...,447.8,96897.7,1.0,0.0300,0.0,0.0,0.0,0.0,1.0,0.216332


In [163]:
def label(column):
    if column['pred_label'] > 0.5:
        #rint ('yes')
        column['pred_label'] = 1
    else:
        column['pred_label'] = 0
    return column
x_test = x_test.apply(label,axis = 1)
x_test.head()        

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label
6765,4.0,15.0,3.0,0.0,0.0,0.0,36.0,21.0,0.0,6.0,...,289.9,84741.8,1.0,0.0615,0.0,0.0,0.0,0.0,1.0,0
13767,9.0,6.0,0.0,0.0,0.0,0.0,15.0,138.0,0.0,0.0,...,140.6,31869.3,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0
9672,6.0,12.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,...,264.0,52674.1,1.0,0.0325,0.0,0.0,0.0,0.0,1.0,0
9116,8.0,12.0,0.0,0.0,0.0,0.0,15.0,96.0,3.0,0.0,...,418.3,90099.3,1.0,0.0209,0.0,0.0,0.0,0.0,1.0,1
10055,8.0,3.0,0.0,0.0,0.0,0.0,0.0,21.0,3.0,0.0,...,447.8,96897.7,1.0,0.0300,0.0,0.0,0.0,0.0,1.0,0


In [165]:
x_test['true_label'] = y_test
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label,true_label
6765,4.0,15.0,3.0,0.0,0.0,0.0,36.0,21.0,0.0,6.0,...,84741.8,1.0,0.0615,0.0,0.0,0.0,0.0,1.0,0,0.0
13767,9.0,6.0,0.0,0.0,0.0,0.0,15.0,138.0,0.0,0.0,...,31869.3,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0,0.0
9672,6.0,12.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,...,52674.1,1.0,0.0325,0.0,0.0,0.0,0.0,1.0,0,0.0
9116,8.0,12.0,0.0,0.0,0.0,0.0,15.0,96.0,3.0,0.0,...,90099.3,1.0,0.0209,0.0,0.0,0.0,0.0,1.0,1,0.0
10055,8.0,3.0,0.0,0.0,0.0,0.0,0.0,21.0,3.0,0.0,...,96897.7,1.0,0.0300,0.0,0.0,0.0,0.0,1.0,0,0.0


In [166]:
#x_test users = x_test[['user_id', 'sku_id', 'cate']].copy()
x_test['user_id'] = users['user_id']
x_test['sku_id'] = users['sku_id']
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label,true_label,user_id,sku_id
6765,4.0,15.0,3.0,0.0,0.0,0.0,36.0,21.0,0.0,6.0,...,0.0615,0.0,0.0,0.0,0.0,1.0,0,0.0,289984.0,70167.0
13767,9.0,6.0,0.0,0.0,0.0,0.0,15.0,138.0,0.0,0.0,...,0.0000,0.0,0.0,1.0,0.0,0.0,0,0.0,291939.0,16139.0
9672,6.0,12.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,...,0.0325,0.0,0.0,0.0,0.0,1.0,0,0.0,288632.0,22140.0
9116,8.0,12.0,0.0,0.0,0.0,0.0,15.0,96.0,3.0,0.0,...,0.0209,0.0,0.0,0.0,0.0,1.0,1,0.0,267503.0,154636.0
10055,8.0,3.0,0.0,0.0,0.0,0.0,0.0,21.0,3.0,0.0,...,0.0300,0.0,0.0,0.0,0.0,1.0,0,0.0,258492.0,39425.0


In [186]:
# 所有购买用户
all_user_set = x_test[x_test['true_label']==1]['user_id'].unique()
print (len(all_user_set))
# 所有预测购买的用户
all_user_test_set = x_test[x_test['pred_label'] == 1]['user_id'].unique()
print (len(all_user_test_set))
all_user_test_item_pair = x_test[x_test['pred_label'] == 1]['user_id'].map(str) + '-' + x_test[x_test['pred_label'] == 1]['sku_id'].map(str)
all_user_test_item_pair = np.array(all_user_test_item_pair)
print (len(all_user_test_item_pair))
#print (all_user_test_item_pair)

126
225
246


In [187]:
pos, neg = 0,0
for user_id in all_user_test_set:
    if user_id in all_user_set:
        pos += 1
    else:
        neg += 1
all_user_acc = 1.0 * pos / ( pos + neg)
all_user_recall = 1.0 * pos / len(all_user_set)
print ('所有用户中预测购买用户的准确率为 ' + str(all_user_acc))
print ('所有用户中预测购买用户的召回率' + str(all_user_recall))

所有用户中预测购买用户的准确率为 0.5333333333333333
所有用户中预测购买用户的召回率0.9523809523809523


In [190]:
#所有实际商品对
all_user_item_pair = x_test[x_test['true_label']==1]['user_id'].map(str) + '-' + x_test[x_test['true_label']==1]['sku_id'].map(str)
all_user_item_pair = np.array(all_user_item_pair)
#print (len(all_user_item_pair))
#print(all_user_item_pair)
pos, neg = 0, 0
for user_item_pair in all_user_test_item_pair:
    #print (user_item_pair)
    if user_item_pair in all_user_item_pair:
        pos += 1
    else:
        neg += 1
all_item_acc = 1.0 * pos / ( pos + neg)
all_item_recall = 1.0 * pos / len(all_user_item_pair)
print ('所有用户中预测购买商品的准确率为 ' + str(all_item_acc))
print ('所有用户中预测购买商品的召回率' + str(all_item_recall))
F11 = 6.0 * all_user_recall * all_user_acc / (5.0 * all_user_recall + all_user_acc)
F12 = 5.0 * all_item_acc * all_item_recall / (2.0 * all_item_recall + 3 * all_item_acc)
score = 0.4 * F11 + 0.6 * F12
print ('F11=' + str(F11))
print ('F12=' + str(F12))
print ('score=' + str(score))

所有用户中预测购买商品的准确率为 0.556910569105691
所有用户中预测购买商品的召回率0.9513888888888888
F11=0.5755395683453236
F12=0.7413419913419912
score=0.6750210221433242
